<a href="https://colab.research.google.com/github/alexaziegler/synthetic-data-for-arr-reporting/blob/main/synthetic_data_for_ARR_reporting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
pip install -U gretel_client pyarrow

  Using cached pyarrow-20.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)


In [20]:
from gretel_client.navigator_client import Gretel
import gretel_client.data_designer.params as P
import gretel_client.data_designer.columns as C
gretel = Gretel(api_key="prompt")

Gretel API Key: ··········
Logged in as alexaziegler01@gmail.com ✅


INFO:gretel_client.navigator_client:Using project: default-sdk-project-5c832794c08e0de
INFO:gretel_client.navigator_client:Project link: https://console.gretel.ai/proj_2yEHgl7oehnfpzJdJmQ6DztnlSx


In [22]:
# Initialize a new Data Designer instance using the `data_designer` factory.
aidd = gretel.data_designer.new(model_suite="apache-2.0")

Creating a unique ID

In [23]:
aidd.add_column(
    C.SamplerColumn(
        name="customer_id",
        type=P.SamplerType.UUID,
        params=P.UUIDSamplerParams(
            prefix="CUST-",  # Optional: adds a prefix
            short_form=True,  # Optional: uses a shorter format
            uppercase=True  # Optional: uses uppercase letters
        )
    )
)

DataDesigner(
    model_suite: apache-2.0
    sampler_columns: ['customer_id']
)

In [24]:
aidd.add_column(
    C.SamplerColumn(
        name="account_vertical",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["Software", "Consulting", "Financial", "Semiconductors", "Cybersecurity","Manufacturing","Education","Government","AI/ML"],
        )
    )
)
aidd.add_column(
    C.SamplerColumn(
        name="account_segment",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["SMB", "Non-Profit", "Lower Mid-Market", "Upper Mid-Market", "Enterprise","Strategic","Partner"],
        )
    )
)

DataDesigner(
    model_suite: apache-2.0
    sampler_columns: ['customer_id', 'account_vertical', 'account_segment']
)

Setting up contract start and end dates (minimum is 1 year, max is 3 year contracts)

In [25]:
aidd.add_column(
    C.SamplerColumn(
        name="contract_start_date",
        type=P.SamplerType.DATETIME,
        params=P.DatetimeSamplerParams(start="2023-01-01", end="2025-12-31")
    )
)
aidd.add_column(
    C.SamplerColumn(
        name="contract_end_date",
        type=P.SamplerType.TIMEDELTA,
        params=P.TimeDeltaSamplerParams(
            dt_min=365,  # Minimum days
            dt_max=1095,  # Maximum days
            reference_column_name="opp_create_date"  # Reference date column
        )
    )
)

DataDesigner(
    model_suite: apache-2.0
    sampler_columns: [
        "customer_id",
        "account_vertical",
        "account_segment",
        "contract_start_date",
        "contract_end_date"
    ]
)

Creating revenue event date

In [26]:
aidd.add_column(
    C.SamplerColumn(
        name="revenue_event_date",
        type=P.SamplerType.DATETIME,
        params=P.DatetimeSamplerParams(start="2023-01-01", end="2025-12-31")
    )
)

DataDesigner(
    model_suite: apache-2.0
    sampler_columns: [
        "customer_id",
        "account_vertical",
        "account_segment",
        "contract_start_date",
        "contract_end_date",
        "revenue_event_date"
    ]
)

Setting up revenue impact column

In [27]:
aidd.add_column(
    C.SamplerColumn(
        name="revenue_impact",
        type=P.SamplerType.UNIFORM,
        params=P.UniformSamplerParams(low=-150000, high=250000),
        data_type="int"  # ✅ Correct parameter name
    )
)



DataDesigner(
    model_suite: apache-2.0
    sampler_columns: [
        "customer_id",
        "account_vertical",
        "account_segment",
        "contract_start_date",
        "contract_end_date",
        "revenue_event_date",
        "revenue_impact"
    ]
)

Setting up LLM field to determine correct event type based on revenue impact column

In [32]:
# Setting up LLM field to determine correct event type based on revenue impact column
aidd.add_column(
   C.LLMTextColumn(
        name="event_type",
        output_type="text",
        model_alias="text",
        prompt="Set this column to Churn or Contraction if {{revenue_impact}} is negative. Set this column to New, Renewal or Expansion if {{revenue_impact}} is positive. This column should only have one value, and the value options are New, Renewal, Expansion, Churn or Contraction depending on {{revenue_impact}}",
        # system_prompt="You are a Revenue Operations analytics professional conducting ARR waterfall analysis.",  # Optional
        # output_format=".." # Removed as it conflicts with output_type="text"
    )
)

# %%
aidd.add_column(
   C.LLMTextColumn(
       name="negative_arr_reason",
       output_type="text",
       model_alias="text",
      prompt="If {{revenue_impact}} is negative, generate a short reason why a company would decrease spend or completely cancel their software contract. For example, the CIO was unhappy with lack of features compared to competitive software offering.",
        # system_prompt="You are a .",  # Optional
        # output_format=".." # Removed as it conflicts with output_type="text"
    )
)

DataDesigner(
    model_suite: apache-2.0
    sampler_columns: [
        "customer_id",
        "account_vertical",
        "account_segment",
        "contract_start_date",
        "contract_end_date",
        "revenue_event_date",
        "revenue_impact"
    ]
    llm_text_columns: ['event_type', 'negative_arr_reason']
)

In [33]:
preview = aidd.preview()

[19:01:19] [INFO] 🚀 Generating preview
[19:01:19] [INFO] 🎲 Step 1: Using samplers to generate 7 columns
[19:01:19] [ERROR] Step 'using-samplers-to-generate-7-columns' failed: UnknownError(1203): An unexpected error occurred. If the error persists on retry attempts, please contact Gretel support. Please check your Workflow config. If the issue persists please contact support.
[19:01:19] [ERROR] 🛑 Something has gone wrong during preview generation. Please inspect the generated data and adjust your configuration as needed.


In [34]:
preview.display_sample_record()

ValueError: No dataset found in the preview results.